In [ ]:
!pip install scikit-surprise

In [ ]:
import pandas as pd
import surprise
import numpy as np

In [ ]:
# 일단 랜덤데이터부터 불러와서 확인해보자.
final_data_random = pd.read_csv('/content/drive/MyDrive/212343_final_data_random.csv', index_col=0)

In [ ]:
final_data_random

,userId,movieId,rating,timestamp
0,199490,2455,3.0,974759178
1,278741,63,3.0,845984922
2,93731,2890,4.0,943835681
3,131404,1347,0.5,1273463851
4,138512,4226,3.5,1142872880
...,...,...,...,...
1007879,212343,189885,2.5,1529431119
1007880,212343,191157,3.5,1531805008
1007881,212343,192659,2.0,1534733035
1007882,212343,192917,2.5,1535607481


In [ ]:
# 영화 제목 확인 위해 영화 전체데이터(원본)를 한번 더 불러오자.
movie_df = pd.read_csv('/content/drive/MyDrive/movies.csv')
movie_df['genres'] = movie_df['genres'].str.split("|")
movie_df

,movieId,title,genres
0,1,Toy Story (1995),"[Adventure, Animation, Children, Comedy, Fantasy]"
1,2,Jumanji (1995),"[Adventure, Children, Fantasy]"
2,3,Grumpier Old Men (1995),"[Comedy, Romance]"
3,4,Waiting to Exhale (1995),"[Comedy, Drama, Romance]"
4,5,Father of the Bride Part II (1995),[Comedy]
...,...,...,...
58093,193876,The Great Glinka (1946),[(no genres listed)]
58094,193878,Les tribulations d'une caissière (2011),[Comedy]
58095,193880,Her Name Was Mumu (2016),[Drama]
58096,193882,Flora (2017),"[Adventure, Drama, Horror, Sci-Fi]"


##1

In [ ]:
# 서프라이즈 라이브러리에서 우리가 필요한 것들을 import하자. 그리고 train, testset으로 한번 나눠보자.
from surprise import SVD, Dataset, accuracy
from surprise.model_selection import train_test_split
from surprise.dataset import DatasetAutoFolds
from surprise import Reader

col = 'user item rating'
reader = Reader(line_format=col, sep='\t') # 반드시 사용자-아이템-평점 순서로
data = Dataset.load_from_df(final_data_random[['userId', 'movieId', 'rating']], reader=reader)
trainset, testset = train_test_split(data, test_size=0.25, random_state=42)

In [ ]:
# SVD를 이용한 잠재요인 협업 필터링을 해보자. KNN은 램이 터지려고 하더라...
algo = SVD() # default epoch : 20 / n_factor: 100
algo.fit(trainset) # 개발자가 __resp__ 안줘서 객체로만 나옴

### 1-1

In [ ]:
# 212343이 안 본 영화를 212343이 봤을 때 몇 점의 평점을 부여할까? 그냥 한 번 살펴보자. 사용자 아이디(uid), 아이템 아이디(iid)는 문자열로 입력해야한다.
uid = str(212343)
iid = str(6)

# 추천 예측 평점
pred = algo.predict(uid, iid)
pred

Prediction(uid='212343', iid='6', r_ui=None, est=3.5789667594022063, details={'was_impossible': False})

In [ ]:
# 정확도 오차는 어느 정도 되는지 확인해보자. 테스트 세트 전체 대상으로.
predictions = algo.test(testset)

print('prediction type :',type(predictions), ' size:',len(predictions))
print('prediction 결과의 최초 5개 추출')

predictions[:5]

prediction type : <class 'list'>  size: 251971
prediction 결과의 최초 5개 추출


[Prediction(uid=117096, iid=151, r_ui=4.0, est=3.7758362893478212, details={'was_impossible': False}),
 Prediction(uid=233811, iid=367, r_ui=3.0, est=2.50081113412467, details={'was_impossible': False}),
 Prediction(uid=133739, iid=3448, r_ui=4.0, est=3.4604568226446997, details={'was_impossible': False}),
 Prediction(uid=19913, iid=349, r_ui=3.5, est=4.229937669515227, details={'was_impossible': False}),
 Prediction(uid=217446, iid=1438, r_ui=3.5, est=2.851744850468152, details={'was_impossible': False})]

In [ ]:
# 오... 나쁘지 않은듯
accuracy.rmse(predictions)

RMSE: 0.9027


0.9027436706418387

##2

In [ ]:
# 이제 최적의 파라미터를 찾은 후 그렇게 분석했을 때 어떻게 결과가 나오는지 살펴보자. 근데 얘는 시간이 좀 오래걸림 ㅋㅋ 너무 파라미터를 많이 넣지 말자.
from surprise.model_selection import GridSearchCV

param_grid = {'n_epochs':[20, 40, 60], 'n_factors':[50, 100, 200]}

# 얘는 인자에 알고리즘 자체를 넣어줌. 첫 번째 인자로 SVD를 넣어주자.
grid = GridSearchCV(SVD, param_grid=param_grid, measures=['rmse', 'mae'], cv=3) # 교차검증은 3번. 더 할 수도 있는데 시간이...
# MAE : Mean Absolute Error, 모델의 예측값과 실제값의 차이의 절대값의 평균, 절대값을 취하기 때문에 가장 직관적으로 알 수 있는 지표이다.(해석에 용이하다.)

grid.fit(data)

print(grid.best_score['rmse'])
print(grid.best_params['rmse'])

0.8984066874427162
{'n_epochs': 20, 'n_factors': 50}


In [ ]:
# 에포크 20, 잠재요인 50개가 최적의 파라미터라고 한다.
# 그럼 이제부터 212343에게 영화를 추천해보자. 전체 데이터를 학습시켜야 하는데, 아래 데이터를 읽는 부분에서 header와 index가 없어야하므로 제거해주고 저장하자.
final_data_random.to_csv('final_data_random_212343_header_none.csv', index=False, header=False)

In [ ]:
col = 'user item rating'
reader = Reader(line_format=col, sep=',', rating_scale=(0.5, 5))
# 전체 데이터를 학습시키기 위해 DatasetAutoFolds를 씀. 얘는 파일 경로를 다시 줘야하니 참고.
data_folds = DatasetAutoFolds(ratings_file='/content/final_data_random_212343_header_none.csv', reader=reader)
data_folds

In [ ]:
# 전체 데이터를 학습 데이터로 생성함.
trainset = data_folds.build_full_trainset()

In [ ]:
# 최적의 파라미터로 학습해보자. 물론 SVD의 최적 파라미터이다. 에포크 20은 디폴트 값이라 안줘도 되긴 하는데, 여기서는 그냥 줬다.
algo = SVD(n_epochs =  20, n_factors = 50)
algo.fit(trainset)

### 2-1

In [ ]:
# 아까처럼 212343이 안 본 6번 영화 데이터에 대해 예측 평점을 다시 구해보자. 영화 제목도 확인.
uid = str(212343)
iid = str(6)

pred = algo.predict(uid, iid, verbose=True)
print(movie_df[movie_df['movieId']==6])

user: 212343     item: 6          r_ui = None   est = 3.02   {'was_impossible': False}
   movieId        title                     genres
5        6  Heat (1995)  [Action, Crime, Thriller]


## 3

In [ ]:
# 위에랑 비교해서 예측 평점이 많이 줄었다. 물론 최적 파라미터로 학습한 이 데이터가 더 정확할 것이다.
# 이제 위에서 뽑아온 랜덤 데이터 중 212343이 안 본 전체 영화데이터를 뽑아와본 후, 예측 평점 순서대로 영화를 추천해보자.
def get_unwatched_movie(final_data_random, movie_df, userId):
    # 입력값으로 들어온 userId가 본 영화의 movieId를 리스트로
    watched_movies = final_data_random[final_data_random['userId']==userId]['movieId'].tolist()

    # 모든 영화의 movieId를 리스트로
    total_movies = movie_df['movieId'].tolist()

    # 모든 영화 중 userId에 해당하는 유저가 본 영화를 제외한 영화를 리스트로
    not_watched_movies = [movie for movie in total_movies if movie not in watched_movies]

    print('모든 영화 수 : ',len(total_movies), '평점 매긴 영화 수 : ',len(watched_movies), '추천 대상 영화 수 : ',len(not_watched_movies))

    return not_watched_movies

In [ ]:
# 212343이 안 본 영화 수 확인
not_watched_movies = get_unwatched_movie(final_data_random, movie_df, 212343)

모든 영화 수 :  58098 평점 매긴 영화 수 :  7884 추천 대상 영화 수 :  50214


In [ ]:
# 위 영화를 대상으로 예측 평점이 높은 순으로 추천 데이터 셋을 만들어보자.
def recommend_movies_for_user(algo, userId, not_watched_movies, top_n):
    predictions = [algo.predict(str(userId), str(movieId)) for movieId in not_watched_movies]

    # 위의 prediction은 예측값을 가지고 있음. 그 예측값으로 정렬하기 위한 함수를 하나 정의하자.
    def sort_est(pred):
        return pred.est
    
    # 예측값을 내림차순 정렬하자.
    predictions.sort(key=sort_est, reverse=True)

    # 상위 top_n개의 prediction 객체
    top_predictions = predictions[:top_n]

    print(f"{userId}님의 인생을 바꿀 Top {top_n} 추천 영화 리스트")

    for pred in top_predictions:
        
        movie_id = int(pred.iid)
        movie_title = movie_df[movie_df["movieId"] == movie_id]["title"].tolist()
        movie_genre = movie_df[movie_df["movieId"] == movie_id]["genres"].tolist()
        movie_rating = pred.est
        
        print(f"{movie_title}: {movie_rating:.2f} // {movie_genre}")

### 3-1

In [ ]:
# 돌려보자.
# ... 나름 인생을 바꿀 영화인데 평점이 좀 짜지 않냐...?
recommend_movies_for_user(algo, 212343, not_watched_movies, 5)

212343님의 인생을 바꿀 Top 5 추천 영화 리스트
['Shoah (1985)']: 3.50 // [['Documentary', 'War']]
['Come and See (Idi i smotri) (1985)']: 3.46 // [['Drama', 'War']]
['Black Mirror: White Christmas (2014)']: 3.46 // [['Drama', 'Horror', 'Mystery', 'Sci-Fi', 'Thriller']]
['Harry Potter and the Deathly Hallows: Part 2 (2011)']: 3.44 // [['Action', 'Adventure', 'Drama', 'Fantasy', 'Mystery', 'IMAX']]
['Enigma of Kaspar Hauser, The (a.k.a. Mystery of Kaspar Hauser, The) (Jeder für sich und Gott Gegen Alle) (1974)']: 3.42 // [['Crime', 'Drama']]


In [ ]:
# 랜덤만 돌려봤는데, 이번엔 앞에서 장르별로 정리한 전체 데이터 대상으로 한번 학습시켜보자. 데이터 개수가 몇개냐고? 1000만개다...
final_data = pd.read_csv('/content/drive/MyDrive/212343_final_data.csv', index_col=0)
final_data

,userId,movieId,rating,timestamp
0,183750,6,3.0,857915212
1,277081,6,5.0,1097520094
2,242725,6,3.5,1275235314
3,231078,6,5.0,1114780711
4,38333,6,5.0,1251843253
...,...,...,...,...
11315851,212343,189885,2.5,1529431119
11315852,212343,191157,3.5,1531805008
11315853,212343,192659,2.0,1534733035
11315854,212343,192917,2.5,1535607481


In [ ]:
# 똑같이 전체 데이터를 학습시켜보자. 학습 시키기 전에 index랑 header 날리기
final_data.to_csv('final_data_212343_header_none.csv', index=False, header=False)

In [ ]:
# 똑같이 fold
col = 'user item rating'
reader = Reader(line_format=col, sep=',', rating_scale=(0.5, 5))
data_folds_all = DatasetAutoFolds(ratings_file='/content/final_data_212343_header_none.csv', reader=reader)
data_folds_all

In [ ]:
all_trainset = data_folds_all.build_full_trainset()

In [ ]:
# 최적의 파라미터로 학습. 물론 SVD
algo = SVD(n_epochs =  20, n_factors = 50)
algo.fit(all_trainset)

In [ ]:
# 212343이 안 본 영화 수 확인. 어차피 안본 영화 수는 랜덤데이터나 전체데이터나 똑같다.
not_watched_movies_all = get_unwatched_movie(final_data, movie_df, 212343)

모든 영화 수 :  58098 평점 매긴 영화 수 :  7884 추천 대상 영화 수 :  50214


### 3-2

In [ ]:
# 돌려보자. 랜덤 100만개와 같은 영화가 하나 있다. 위 결과와 예측 평점은 0.1정도 차이나는듯.
recommend_movies_for_user(algo, 212343, not_watched_movies_all, 5)

''' 

실제 IMDB에 등록되어있는 아래 영화 평점
1. Civil War, The (1990) // Rating: 9.1/10 · ‎16,926 votes
2. Revenge of the Pink Panther (1978) // Rating: 6.6/10 · ‎22,171 votes
3. The Invisible Guest (2016) // Rating: 8/10 · ‎179,344 votes
4. Black Mirror: White Christmas (2014) // Rating: 9.1/10 · ‎59,458 votes
5. Pink Panther Strikes Again, The (1976) // Rating: 7.2/10 · ‎30,449 votes

'''

212343님의 인생을 바꿀 Top 5 추천 영화 리스트
['Civil War, The (1990)']: 4.13 // [['Documentary', 'War']]
['Revenge of the Pink Panther (1978)']: 3.65 // [['Comedy', 'Crime']]
['The Invisible Guest (2016)']: 3.63 // [['Thriller']]
['Black Mirror: White Christmas (2014)']: 3.59 // [['Drama', 'Horror', 'Mystery', 'Sci-Fi', 'Thriller']]
['Pink Panther Strikes Again, The (1976)']: 3.55 // [['Comedy', 'Crime']]


##4

In [ ]:
# 그럼 문득 궁금할 것이다. 이 예측 모델이 제대로 점수를 주는걸까? 얘가 평점 3.5점을 준 영화와 5점을 준 영화 각각 하나만 지우고 다시 학습을 시켜보자. 그래놓고 비슷하게 별점을 주는지 보자.
for_delete = final_data[final_data['userId'] == 212343]
for_delete = for_delete[(for_delete['movieId'] == 177901)|(for_delete['movieId'] == 50)]
for_delete

,userId,movieId,rating,timestamp
11308007,212343,50,5.0,1436133986
11315820,212343,177901,3.5,1516509756


In [ ]:
final_data_2 = final_data[~(final_data['movieId'].isin(for_delete['movieId']) & final_data['userId'].isin([212343]))]
final_data_2

,userId,movieId,rating,timestamp
0,183750,6,3.0,857915212
1,277081,6,5.0,1097520094
2,242725,6,3.5,1275235314
3,231078,6,5.0,1114780711
4,38333,6,5.0,1251843253
...,...,...,...,...
11315851,212343,189885,2.5,1529431119
11315852,212343,191157,3.5,1531805008
11315853,212343,192659,2.0,1534733035
11315854,212343,192917,2.5,1535607481


In [ ]:
# 재학습 들어갑니다잉
final_data_2.to_csv('final_data_2_212343', index=False, header=False)

In [ ]:
col = 'user item rating'
reader = Reader(line_format=col, sep=',', rating_scale=(0.5, 5))
data_folds_2 = DatasetAutoFolds(ratings_file='/content/final_data_2_212343', reader=reader)
data_folds_2

In [ ]:
trainset_2 = data_folds_2.build_full_trainset()

In [ ]:
algo = SVD(n_epochs =  20, n_factors = 50, random_state = 0)
algo.fit(trainset_2)

In [ ]:
# 이번엔 아까 지웠던 영화 데이터만 가져오자. 그냥 50번, 177901번 가져오면 되긴 하지만... 더 많이 지웠다면 이렇게 가져오면 됨.
not_watched_movies = for_delete['movieId'].tolist()
not_watched_movies

[50, 177901]

In [ ]:
# 예측값이 짜긴한데... 오차 감안하면 그냥 비슷하게 주는듯? 5 vs 3.05 // 3.5 vs 2.34
recommend_movies_for_user(algo, 212343, not_watched_movies, len(not_watched_movies))

212343님의 인생을 바꿀 Top 2 추천 영화 리스트
['Usual Suspects, The (1995)']: 3.05 // [['Crime', 'Mystery', 'Thriller']]
['The Guilty (2000)']: 2.34 // [['Crime', 'Drama', 'Thriller']]
